In [1]:
from operator import attrgetter
from sortedcontainers import SortedList
import bisect
import math
import random as random
from pyroaring import BitMap
from multiprocessing import Process, Queue

def binarysearch(a, x):
    i = bisect.bisect_left(a, x)
    if i != len(a) and a[i] == x:
        #print(a[i], x)
        return True
    else:
        return False

# Welcome.

In this notebook there are 3 algorithms:

* Roaring Teleport Lists
* Roaring Split List
* Split List

**Roaring** refers to the usage of [Roaring Bitmaps](https://arxiv.org/pdf/1603.06549.pdf) as indexes.

That are compared with `sortedcontainers`'s SortedList.

## Invariants

All data structures have the following invariants:

* Geometrically distributed random heights that hold both the data and the indexes
* Clever use of min and max values in order to speed up lookups and inserts

### Roaring MinMaxDict

This is the container that will hold both the indexes and the data within each "height".

In [2]:
class RoaringMinMaxDict(dict):
    def __init__(self, *arg, **kwargs):
        super().__init__()
        self.indexes = BitMap()
        self.max = float('-inf')
        self.min = float('inf')

    def insert(self, key, value=None):
        self.indexes.add(key)
        self.max = self.indexes.max()
        self.min = self.indexes.min()
        dict.__setitem__(self, key, value)

    ## Deletes a VALUE
    def delete(self, key):
        self[key] = '<deleted>'

    ## Discards a VALUE/wipes it out of the index and dict
    def discard(self, key):
        self.pop(key)
        self.indexes.discard(key)
        self.max = self.indexes.max()
        self.min = self.indexes.min()

    def __lt__(self, other):
        return self.indexes.min() < other

## Teleport List

This is the Teleport list, the fastest data structure we have for inserts and deletions.

In [3]:
class TeleportList:

    def __init__(self):
        self.height = -1
        self.subdicts = []

    def insert(self, key, value=None):

        height = int(-(math.log2(random.random())))

        if self.height < height:
            for i in range(height - self.height):
                self.subdicts.append(RoaringMinMaxDict())
            self.height = height

        highest = self.subdicts[height]

        if key not in highest.indexes:
            highest.insert(key, value)

    def lookup(self, key):

        for i in self.subdicts:
            if i.min <= key <= i.max:
                if key in i.indexes:
                    return i[key]
        return False

    def delete(self, key):
        for i in self.subdicts:
            if i.min <= key <= i.max:
                if key in i.indexes:
                    i.delete(key)

    def discard(self, key):
        for i in self.subdicts:
            if i.min <= key <= i.max:
                if key in i.indexes:
                    i.discard(key)

    def show_hedges(self):
        for i in self.subdicts:
            print(i.indexes)

    def show_minmax(self):
        for i in self.subdicts:
            print(f'({i.min}, {i.max})')

### Roaring MaxDict

This is the container that will hold both the indexes and the data within each "height".

In [4]:
class RoaringMaxDict(dict):
    def __init__(self, nr):
        super().__init__()
        self.indexes = BitMap()
        self.indexes.add(nr)
        self.max = float("-inf")

    def insert(self, key, value=None):
        self.indexes.add(key)
        self.max = self.indexes.max()
        dict.__setitem__(self, key, value)

    def delete(self, key):
        self[key] = '<deleted>'

    def __lt__(self, other):
        if isinstance(other, int):
            return self.max < other
        else:
            return self.max < other.max

class SortableSubList:
    def __init__(self):
        self.sublists = []

## Split List

This is the Split list, the most stable data structure we have.

In [5]:
def splitter_two(arr, load):
    half = load // 2
    zs = arr[0:half]
    arr = arr.difference(zs)
    return zs

# if overload is detected, splits and adds a new split into levellist
def Overload(blist, i, load):
    B = RoaringMaxDict(5)
    candidate_sublist = blist.sublists[i]
    B.indexes = splitter_two(candidate_sublist.indexes, load)
    B.max = B.indexes.max()
    bisect.insort_left(blist.sublists, B)

class RoaringSplitList:
    def __init__(self):
        self.height = -1
        self.blists = []
        self.load = 2000

    def lookup(self, nr):
        for he in self.blists:
            if nr <= he.sublists[-1].max:
                i = bisect.bisect_left(he.sublists, nr)
                if  i != len(he.sublists) and he.sublists[i].indexes[0] <= nr:
                    if nr in he.sublists[i].indexes:
                        return True
        print(nr) #can be used to detect if the search works or not
        return nr

    def insert(self, nr):
        ## Getting the estimated geometric distribution
        height = int(-(math.log2(random.random())))
        ## Checking whether we need to add new edges
        if self.height < height:
            for i in range(height - self.height):
                B = SortableSubList()
                C = RoaringMaxDict(nr)
                B.sublists.append(C)
                self.blists.append(B)
            self.height = height

        ## Getting the to-be-added list
        blist = self.blists[height]

        L = len(blist.sublists)
        i = bisect.bisect_left(blist.sublists, nr)

        if i == 0 or L == 1:
            updated_maxlist = blist.sublists[0]
            updated_maxlist.insert(nr)
            if len(updated_maxlist.indexes) == self.load:
                    Overload(blist, 0, self.load)
        elif i == L:
            updated_maxlist = blist.sublists[-1]
            updated_maxlist.insert(nr)
            if len(updated_maxlist.indexes) == self.load:
                    Overload(blist, i-1, self.load)

        else:
            updated_maxlist = blist.sublists[i]
            if updated_maxlist.indexes[0] <= nr:
                updated_maxlist.indexes.add(nr)
                if len(blist.sublists[i].indexes) == self.load:
                    Overload(blist, i, self.load)
            else:
                updated_maxlist = blist.sublists[i-1]
                updated_maxlist.insert(nr)
                if len(updated_maxlist.indexes) == self.load:
                    Overload(blist, i-1, self.load)

    def show_hedges(self):
        for i in self.blists:
            maxes = [j.max for j in i.sublists]
            print(maxes)

    def show_edges(self):
        for i in self.blists:
            print("--------" + str(len(i.sublists)) +"-----------")
            for j in i.sublists:
                print(j.indexes)

    def show_minmax(self):
        for i in self.blists:
            print(f'({i.min}, {i.max})')

## Split List

In [6]:
#splits the overloaded list into two consecutive parts
def splitterSimple(arr, load):
    half = load // 2
    zs = [0] * half
    for i in range(half-1, -1, -1):
        zs[i] = arr.pop()
    return zs

# if overload is detected, splits and adds a new split into levellist
def OverloadSimple(blist, i, load):
    B = IntervalList(5)
    candidate_sublist = blist.sublists[i]
    B.indexes = splitterSimple(candidate_sublist.indexes, load)
    #B.i = - blist.sublists[i].i - 1
    B.max = candidate_sublist.max
    candidate_sublist.max = candidate_sublist.indexes[-1]
    blist.sublists.insert(i+1, B)

class IntervalList:
    def __init__(self, nr):
        self.indexes = [nr]
        self.max = float("-inf")

    def __lt__(self, other):
        if isinstance(other, int):
            return self.max < other
        else:
            return self.max < other.max

class LevelList:
    def __init__(self):
        self.sublists = []
        self.min = float("inf")
        self.max = float("-inf")

    def __lt__(self, other):
        return self.max > other

class SplitList:# Rucy, rename it!
    def __init__(self):
        self.height = -1
        self.blists = []
        self.load = 2000


    def lookup(self, nr):

        for he in self.blists:

            if nr <= he.sublists[-1].max:
                i = bisect.bisect_left(he.sublists, nr)

                if  i != len(he.sublists) and he.sublists[i].indexes[0] <= nr:
                    if binarysearch(he.sublists[i].indexes, nr):
                        return True
        #print(nr) #can be used to detect if the search works or not
        return nr

    def insert(self, nr):
        ## Getting the estimated geometric distribution
        height = int(-(math.log2(random.random())))
        ## Checking whether we need to add new edges
        if self.height < height:
            for i in range(height - self.height):
                B = LevelList()
                C = IntervalList(nr)
                C.max = -1 #arbitrary contemporary max
                B.sublists.append(C)
                self.blists.append(B)
            self.height = height

        ## Getting the to-be-added list
        blist = self.blists[height]

        ## Doing the search to see which Intervallist it should be in
        L = len(blist.sublists)
        i = bisect.bisect_left(blist.sublists, nr)

        ## If it's smaller than all other elements then just insort it
        if i == 0 or L == 1:
            candid = blist.sublists[0]
            bisect.insort_left(candid.indexes, nr)
            candid.max = candid.indexes[-1]
            if len(candid.indexes) == self.load:
                    OverloadSimple(blist, 0, self.load)
        ## If it's bigger than all the other elements than just append it
        elif i == L:
            candid = blist.sublists[-1]
            candid.indexes.append(nr)
            candid.max = nr
            if len(candid.indexes) == self.load:
                    OverloadSimple(blist, i-1, self.load)

            ## Else add it
        else:
            candidate_sublist = blist.sublists[i]
            # if the element is also bigger than the minimum of the current list than we insort it
            if candidate_sublist.indexes[0] <= nr:
                bisect.insort_left(candidate_sublist.indexes, nr)

                if len(blist.sublists[i].indexes) == self.load:
                    OverloadSimple(blist, i, self.load)
            # then the element must be smaller then the min of the current list but therefore
            # bigger than the max of the previous list-- so we just append it
            else:
                candidate_sublist = blist.sublists[i-1]
                candidate_sublist.indexes.append(nr)
                candidate_sublist.max = nr

                if len(candidate_sublist.indexes) == self.load:
                    OverloadSimple(blist, i-1, self.load)

    def show_hedges(self):
        for i in self.blists:
            maxes = [j.max for j in i.sublists]
            print(maxes)

    def show_edges(self):
        for i in self.blists:
            print("--------" + str(len(i.sublists)) +"-----------")
            for j in i.sublists:
                print(j.indexes)

    def show_minmax(self):
        for i in self.blists:
            print(f'({i.min}, {i.max})')

# Benchmarks.

Let's do some simple ones.

Insert and lookup of 1 million elements.

In [11]:
random.seed(0)

tlist = TeleportList()
rslist = RoaringSplitList()
slist = SortedList()
splist = SplitList()

nr = 1000000

ten_thousand_integers = [random.randint(1, 2000000) for i in range(nr)]

def insert_tlist(tl):
    for i in range(nr):
        tl.insert(ten_thousand_integers[i])

def insert_rslist(rtl):
    for i in range(nr):
        rtl.insert(ten_thousand_integers[i])

def insert_slist(sl):
    for i in range(nr):
        sl.add(ten_thousand_integers[i])

def insert_nlist(novus):
    for i in range(nr):
        novus.insert(ten_thousand_integers[i])

%timeit -r 1 -n 1 insert_tlist(tlist)
%timeit -r 1 -n 1 insert_rslist(rslist)
%timeit -r 1 -n 1 insert_slist(slist)
%timeit -r 1 -n 1 insert_nlist(splist)

1.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1.87 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2.82 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
3.79 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
def lookup_tlist(tl):
    for i in range(nr):
        tl.lookup(ten_thousand_integers[i])

def lookup_rslist(rtl):
    for i in range(nr):
        rtl.lookup(ten_thousand_integers[i])

def lookup_slist(sl):
    for i in range(nr):
        ten_thousand_integers[i] in sl

def lookup_nlist(novus):
    for i in range(nr):
        novus.lookup(ten_thousand_integers[i])

%timeit -r 10 -n 1 lookup_tlist(tlist)
%timeit -r 10 -n 1 lookup_rslist(rslist)
%timeit -r 10 -n 1 lookup_slist(slist)
%timeit -r 10 -n 1 lookup_nlist(splist)

## Benchmarking Parallelized SplitList

> vs regular SplitList

Make sure to **restart the kernel** if you made any changes to the parallelized SplitList class.

In [11]:
from splitlist_parallel import SplitList as SplitListParallelized
from splitlist_parallel2 import SplitList as SplitListParallelized2

# inserts
def test_insert(lst, reps):
    for i in range(reps):
        lst.insert(ten_thousand_integers[i])
        

# lookups
def test_lookup(lst, reps):
    for i in range(reps):
        lst.lookup(ten_thousand_integers[i])

In [12]:
random.seed(0)

split_list = SplitList()
split_list_parallel = SplitListParallelized()
split_list_parallel2 = SplitListParallelized2()

nr = 1_000_000
ten_thousand_integers = [random.randint(1, 2000000) for i in range(nr)]

In [13]:
# Test insert

%timeit -r 1 -n 1 test_insert(split_list, nr)
%timeit -r 1 -n 1 test_insert(split_list_parallel, nr)
%timeit -r 1 -n 1 test_insert(split_list_parallel2, nr)

3.84 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
4.02 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
4.04 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [14]:
# Test lookup
lookups = 50

%timeit -r 10 -n 1 test_lookup(split_list, lookups)
%timeit -r 10 -n 1 test_lookup(split_list_parallel, lookups)
%timeit -r 10 -n 1 test_lookup(split_list_parallel2, lookups)

309 µs ± 81.7 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
4.63 s ± 78.6 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


OSError: [Errno 24] Too many open files

Note 368 **micro**-seconds vs 137 **milli**-seconds. Parallelized version is slower by a factor of ~370.

In [18]:
random.seed(0)

split_list = SplitList()
split_list_parallel = SplitListParallelized()

nr = 10_000
ten_thousand_integers = [random.randint(1, 2000000) for i in range(nr)]

In [19]:
# Test insert

%timeit -r 1 -n 1 test_insert_SplitList(nr)
%timeit -r 1 -n 1 test_insert_SplitListParallelized(nr)

22.4 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
22.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [20]:
# Test lookup

lookups = 1000

%timeit -r 10 -n 1 test_lookup_SplitList(lookups)
%timeit -r 10 -n 1 test_lookup_SplitListParallelized(lookups)

4.02 ms ± 1.03 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
622 ms ± 99.2 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


Again, parallelized is slower, this time by ~170.